In [1]:
import os
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
import shutil

In [4]:
import pickle

In [66]:
# torch.load('kaal_extract/TCGA-BA-4074/TCGA-BA-4074_10_1.jpeg.pt')

In [2]:
df23 =  pd.read_csv('g2_g3.csv')
df23_2 = df23.set_index('HNSC')
there = set(list(x[:] for x in df23_2.index))
wsi_there = os.listdir('regular_plip_extracted/')
use = list(there.intersection(wsi_there))
df23_2 = df23_2.loc[use]
df23_2['cluster'] = df23_2['cluster'] -2

df23_3  = df23_2.sample(frac=1)

class1 = list(df23_3[df23_3['cluster']==1].index)
class0 = list(df23_3[df23_3['cluster']==0].index)

from sklearn.model_selection import train_test_split
C1_X_train, C1_X_test = train_test_split(class1, test_size=0.3)
C0_X_train, C0_X_test = train_test_split(class0, test_size=0.3)

C1_X_validate, C1_X_test = train_test_split(C1_X_test, test_size=0.4)
C0_X_validate, C0_X_test = train_test_split(C0_X_test, test_size=0.4)


X_train = [];X_train.extend(C1_X_train);X_train.extend(C0_X_train);
X_test = [];X_test.extend(C1_X_test);X_test.extend(C0_X_test)
X_validate = [];X_validate.extend(C1_X_validate);X_validate.extend(C0_X_validate)

random.shuffle(X_train);
random.shuffle(X_test)
random.shuffle(X_validate);

data_info = {};
data_info['train'] = X_train;data_info['test'] = X_test;data_info['validate'] = X_validate




In [5]:
with open('Datasets/data_info0315_regD.pkl','wb') as f:
    pickle.dump(data_info,f)

In [6]:
# list(df23_2.index)

In [7]:
print(" C1 - Train : {} , Validate : {} , Test : {} ".format(len(C1_X_train),len(C1_X_test),len(C1_X_validate)))
print(" C0 - Train : {} , Validate : {} , Test : {} ".format(len(C0_X_train),len(C0_X_test),len(C0_X_validate)))

 C1 - Train : 70 , Validate : 12 , Test : 18 
 C0 - Train : 59 , Validate : 11 , Test : 15 


In [8]:
data = {};
data['train'] = {};data['test'] = {};data['validate'] = {};
data['train']['X'] = [];data['train']['Y'] = []
data['test']['X'] = [];data['test']['Y'] = []
data['validate']['X'] = [];data['validate']['Y'] = []

for i,pID in enumerate(X_train[:]):
    fol_p = os.path.join('regular_plip_extracted/',pID) 
    tiles = os.listdir(fol_p)
    tile_data = []
    for tile in tiles:
        tile_p = os.path.join(fol_p,tile)
        
        np1 = torch.load(tile_p).numpy()
        # print(np1[0].shape)
        tile_data.append(np1)
        
    data['train']['X'].extend(np.array(tile_data))
    data['train']['Y'].extend(list(df23_3.loc[pID] for each in range(len(tile_data)) ))
    # except:
    #     print('not there {}'.format(pID))

data['train']['X'] = np.array(data['train']['X']);
data['train']['Y'] = np.array(data['train']['Y'])
data['train']['X'] = np.squeeze(data['train']['X'], axis=1)


for i, pID in enumerate(X_validate[:]):
    fol_p = os.path.join('regular_plip_extracted/', pID)
    tiles = os.listdir(fol_p)
    tile_data = []
    for tile in tiles:
        tile_p = os.path.join(fol_p, tile)

        np1 = torch.load(tile_p).numpy()
        tile_data.append(np1)

    data['validate']['X'].extend(np.array(tile_data))
    data['validate']['Y'].extend([df23_3.loc[pID] for each in range(len(tile_data))])

data['validate']['X'] = np.array(data['validate']['X'])
data['validate']['Y'] = np.array(data['validate']['Y'])
data['validate']['X'] = np.squeeze(data['validate']['X'], axis=1)


for i, pID in enumerate(X_test[:]):
    fol_p = os.path.join('regular_plip_extracted/', pID)
    tiles = os.listdir(fol_p)
    tile_data = []
    for tile in tiles:
        tile_p = os.path.join(fol_p, tile)

        np1 = torch.load(tile_p).numpy()
        tile_data.append(np1)

    data['test']['X'].extend(np.array(tile_data))
    data['test']['Y'].extend([df23_3.loc[pID] for each in range(len(tile_data))])

data['test']['X'] = np.array(data['test']['X'])
data['test']['Y'] = np.array(data['test']['Y'])
data['test']['X'] = np.squeeze(data['test']['X'], axis=1)

In [9]:
import pickle

In [10]:
with open('Datasets/data_031524_1_regD.pkl','wb') as f:
    pickle.dump(data,f)

In [11]:
wsi_data = {};
for pID in df23_3.index:
    fol_p = os.path.join('regular_plip_extracted/',pID)
    tiles = os.listdir(fol_p) ;
    tile_data = []
    for tile in tiles:
        tile_p = os.path.join(fol_p,tile);
        tile_data.append(torch.load(tile_p).numpy())
        
    np1 = np.array(tile_data)
    wsi_data[pID] = {} ;
    wsi_data[pID]['tiles'] = np.squeeze(np1,axis=1)
    wsi_data[pID]['class'] = df23_3.loc[pID][0]
    

In [12]:
wsi_data['TCGA-UF-A71E']['tiles'].shape

(833, 768)

In [163]:
# wsi_data.keys()

In [13]:
with open('Datasets/wsi_data_g2_g3_regD.pkl','wb') as f:
    pickle.dump(wsi_data,f)